# 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from sklearn.model_selection import train_test_split
from itertools import permutations
from transformers import AutoTokenizer

In [8]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7351 entries, 0 to 7350
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7351 non-null   object
 1   sentence_0  7351 non-null   object
 2   sentence_1  7351 non-null   object
 3   sentence_2  7351 non-null   object
 4   sentence_3  7351 non-null   object
 5   answer_0    7351 non-null   int64 
 6   answer_1    7351 non-null   int64 
 7   answer_2    7351 non-null   int64 
 8   answer_3    7351 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 517.0+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [4]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1780 non-null   object
 1   sentence_0  1780 non-null   object
 2   sentence_1  1780 non-null   object
 3   sentence_2  1780 non-null   object
 4   sentence_3  1780 non-null   object
dtypes: object(5)
memory usage: 69.7+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3
0,TEST_0000,"자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐...","결정론은 모든 사건이 원인과 결과의 연쇄에 의해 발생한다고 주장하며, 이는 인간의 ...",그러나 인간의 인식과 선택 과정에서 나타나는 복잡성과 예측 불가능성은 자유 의지의 ...,"결국, 자유 의지와 결정론은 서로를 배제하기보다는, 인간 경험의 다양한 측면을 설명..."
1,TEST_0001,사회적 낙인은 개인의 자아 존중감에 부정적인 영향을 미친다.,"건강 불평등은 이러한 낙인으로 인해 더욱 심화되며, 특정 집단이 의료 서비스 접근에...","결국, 사회적 낙인과 건강 불평등은 서로 연결되어 있으며, 이를 해결하기 위한 포괄...","낙인으로 인해 사람들은 사회적 고립을 경험하고, 이는 정신적 및 신체적 건강에 악영..."
2,TEST_0002,글쓰기 능력을 키우기 위해서는 꾸준한 연습이 필수적이다.,"마지막으로, 독서를 통해 다른 작가들의 기법을 배우는 것은 창의력을 자극하는 데 도...",피드백을 받는 과정은 글의 질을 향상시키는 중요한 요소로 작용한다.,다양한 주제에 대해 글을 써보면 자신의 스타일과 강점을 발견할 수 있다.
3,TEST_0003,작은 공간에서도 효율적으로 사용할 수 있어 집안의 혼잡함을 줄여준다.,정기적으로 내용을 점검하면 필요 없는 물건을 정리할 수 있다.,각 칸을 활용하여 카테고리별로 물건을 나누면 찾기 쉬워진다.,다용도 수납함은 다양한 물건을 정리하는 데 유용하다.
4,TEST_0004,음악은 특정 문화의 가치와 전통을 반영하는 중요한 매체이다.,이러한 음악적 표현은 공동체의 소속감을 증진시키는 역할을 한다.,각 문화는 고유한 음악적 요소를 통해 정체성을 형성하고 강화한다.,"결국, 음악은 개인과 집단의 문화적 정체성을 이해하는 데 필수적인 요소로 작용한다."


In [5]:
# 텍스트 정제
def clean_text(text):
  # 특수문자 제거
  text = re.sub(r'[^\w\s]', '', text)
  # 소문자 변환
  text = text.lower()
  # 불필요한 공백 제거
  text = ' '.join(text.split())
  return text

In [9]:
# 정제 전
train['sentence_0'][0]

'블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.'

In [11]:
# 텍스트 정제
for i in range(4):
    train[f'sentence_{i}'] = train[f'sentence_{i}'].apply(clean_text)
    test[f'sentence_{i}'] = test[f'sentence_{i}'].apply(clean_text)

In [12]:
# 정제 후
train['sentence_0'][0]

'블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다'

In [13]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

c:\Users\Yeo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

c:\Users\Yeo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yeo\.cache\huggingface\hub\models--klue--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [14]:
# 순열 조합 생성 및 토크나이징
def tokenize_permuted_sentences(row):
    base = row['sentence_0']
    others = ['sentence_1', 'sentence_2', 'sentence_3']
    permutations_list = permutations(others, 3)  # 3개 문장의 순열

    tokenized_sequences = []
    for perm in permutations_list:
        # 문장들을 순서대로 이어붙임
        sent_list = [base] + [row[p] for p in perm]
        joined_text = " ".join(sent_list)  # 하나의 input으로 만들기
        encoded = tokenizer(
            joined_text,
            padding='max_length',
            truncation=True,
            max_length=256,
            return_tensors='pt'
        )
        tokenized_sequences.append(encoded)

    return tokenized_sequences  # 총 6개

In [15]:
# 각 행에 대해 6가지 순열 조합 토크나이징
train['tokenized_permutations'] = train.apply(tokenize_permuted_sentences, axis=1)
test['tokenized_permutations'] = test.apply(tokenize_permuted_sentences, axis=1)

In [16]:
train['tokenized_permutations'][0]

[{'input_ids': tensor([[    0, 18966,  3726,  2073,  4653,  3747,  2079,  5767,  2047,  2069,
           1750,  2318,  5144,  2067,  2764,  1295,  1513,  2062,  3839,  2470,
           4803,  2073,  7243,  2031,  2170,  2318,  4901,  2138,  3894,  2205,
           2307,  3772,  2125,  3844,  2138,  6627,  2205,  2259,   842,  5291,
           2085,  1295,  1513,  2062,  3731, 11187, 18966,  4568,  2079,  4653,
           4119,  2073,  4646, 19521,  4901,  2085,  1295,  1513,  2259,  3862,
           3828,  2069,  4196,  2085, 10149,  2069, 16954,   544,  4653,  2259,
           5131,  5391,  2183,  2470,  3809,  6233,  6965,  2496,  2051,  6251,
           2079,  3662,  2047,  2069,  7848, 11187,  6202,  4538,     2,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,   

In [17]:
test['tokenized_permutations'][0]

[{'input_ids': tensor([[    0,  3936,  4659,  2522,  3794,  2570,  2073,  4084, 20373,  2205,
           2259,  4453,  6233,  7091,  2118,  3683,  1504,   867,  2079,  9719,
           3662,  2047,  2119,  7650,  2085,  3985,  2116,  1513,  2062,  3794,
           2570,  2073,  3721,  3863,  2052,  4511,  2145,  3731,  2079, 12653,
           2170,  4249,  3888,  4538,  2088,  3831,  2205,  2307,  1504,  2259,
           3804,  2079,  4151,  6509,  4121,  2651,  1295,  1513,  2062,  3678,
           3804,  2079,  4263,  2145,  4025,  3747, 27135,  4621,  2259,  5629,
           2047,  2145,  5501,  5391,  2183,  2047,  2073,  3936,  4659,  2079,
           3990,  2138,  6305,  4538,  3983,  3936,  4659,  2522,  3794,  2570,
           2073,  4084,  2138,  6337, 31302,  2178,  4000,  3804,  4043,  2079,
           3757,  2470,  4859,  2069,  3788,  2205,  2259,   842,  5291,  2085,
           1295,  1513,  2259,  5230,  6374, 31221,  4453,  6233,  3923,  2651,
           1295,  1513,  2

# 2. 데이터셋 구성

In [ ]:
# 데이터셋 클래스 정의
class SentenceOrderDataset(Dataset):
    """
    문장 쌍을 받아 BERT 입력 형식으로 변환하는 PyTorch Dataset 클래스

    Args:
        texts (List[Tuple[str, str]]): (문장1, 문장2) 형태의 튜플 리스트
        labels (List[int]): 문장 순서가 맞는지 여부를 나타내는 정수 레이블 (예: 0 또는 1)
        tokenizer (transformers.PreTrainedTokenizer): HuggingFace 토크나이저 객체
    """
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text_pair = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text_pair[0],               # 첫 번째 문장
            text_pair[1],               # 두 번째 문장
            padding='max_length',       # 최대 길이에 맞춰 패딩
            truncation=True,            # 잘라도 됨
            max_length=128,
            return_tensors='pt'         # PyTorch 텐서 반환
        )

        # squeeze()는 예외 위험이 있으므로 index 사용
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'token_type_ids': encoding['token_type_ids'][0],  # 문장 구분 ID 추가
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 모델 아키텍쳐

In [ ]:
from transformers import AutoModelForSequenceClassification

class SentenceOrderModel(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.bert = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2  # 순서가 맞는지 아닌지 이진 분류
    )

  def forward(self, input_ids, attention_mask, labels=None):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )
    return outputs

# 4. 학습 코드

In [ ]:
from transformers import Trainer, TrainingArguments

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
   )

# 학습 실행
trainer.train()

In [ ]:
#5. 평가 및 추론
def predict_order(sent1, sent2):
  inputs = tokenizer(
      sent1,
      sent2,
      return_tensors='pt',
      padding=True,
      truncation=True,
      max_length=128
  )

  outputs = model(**inputs)
  predictions = torch.softmax(outputs.logits, dim=1)
  return predictions.argmax().item()